In [ ]:
%matplotlib inline

# Calibration scenario with missing values.


In [ ]:
from __future__ import annotations

from gemseo.algos.parameter_space import ParameterSpace
from gemseo.datasets.dataset import Dataset
from gemseo.disciplines.analytic import AnalyticDiscipline
from numpy import array
from numpy import nan

from gemseo_calibration.scenario import CalibrationMeasure
from gemseo_calibration.scenario import CalibrationScenario

model = AnalyticDiscipline({"y": "a*x", "z": "b*x"}, name="model")

prior = ParameterSpace()
prior.add_variable("a", lower_bound=0.0, upper_bound=10.0, value=0.0)
prior.add_variable("b", lower_bound=0.0, upper_bound=10.0, value=0.0)

data = array([
    [1, 1.0, 2.0, nan],
    [2, 1.0, nan, 3.0],
    [3, 2.0, 4.0, nan],
    [4, 2.0, nan, 6.0],
])
reference_data = Dataset.from_array(
    data,
    variable_names=["index", "x", "y", "z"],
    variable_names_to_group_names={
        "index": "inputs",
        "x": "inputs",
        "y": "outputs",
        "z": "outputs",
    },
).to_dict_of_arrays(False)

control_outputs = [CalibrationMeasure("y", "MSE"), CalibrationMeasure("z", "MSE")]
calibration = CalibrationScenario(model, "x", control_outputs, prior)
calibration.execute({
    "algo": "NLOPT_COBYLA",
    "reference_data": reference_data,
    "max_iter": 100,
})